In [1]:
import numpy as np
import pandas as pd
!pip install wrds
import wrds
import wrds                  
conn = wrds.Connection() 

Enter your WRDS username [root]:keback
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: n
You can create this file yourself at any time
with the create_pgpass_file() function.
Loading library list...
Done


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 54.8 gigabytes of available RAM



# Annual Compustat

An example of pulling data from the annual Compustat table.  datadate is the end of the fiscal year.  We impose standard filters.  See https://wrds-web.wharton.upenn.edu/wrds/demo/demoform_compustat.cfm for a full list of Compustat variable definitions. 

In [3]:
variables = ['txditc','pstkrv','pstkl','pstk','ceq','seq','at','lt','cogs','xsga','xint','sale','revt','mib']            

start = '1963-01-01'

variables = ', '.join(variables)
df = conn.raw_sql(             "SELECT a.gvkey, a.datadate, b.tic,"
                               + variables +
                               " from comp.FUNDA a left outer join comp.Names b "
                               " on a.gvkey = b.gvkey "
                               " where a.datadate >= '" + start + "' "
                               " and INDFMT='INDL' and DATAFMT='STD' and POPSRC='D' and CONSOL='C' "
                               " order by a.gvkey, datadate ", date_cols=['datadate'])
df.gvkey = df.gvkey.astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526495 entries, 0 to 26494
Data columns (total 17 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   gvkey     526495 non-null  int64         
 1   datadate  526495 non-null  datetime64[ns]
 2   tic       526340 non-null  object        
 3   txditc    411620 non-null  float64       
 4   pstkrv    445050 non-null  float64       
 5   pstkl     446643 non-null  float64       
 6   pstk      449108 non-null  float64       
 7   ceq       444379 non-null  float64       
 8   seq       442917 non-null  float64       
 9   at        448949 non-null  float64       
 10  lt        446341 non-null  float64       
 11  cogs      445402 non-null  float64       
 12  xsga      359266 non-null  float64       
 13  xint      400147 non-null  float64       
 14  sale      447243 non-null  float64       
 15  revt      447245 non-null  float64       
 16  mib       432746 non-null  float64     

# Ratios and growth rates

We illustrate some standard types of calculations with Compustat data by computing some Fama-French characteristics.

In [4]:
# we shift fiscal-year-end data to the end of June in the next calendar year, to accommodate the reporting lag
# this shift is standard for Fama and French

df['date'] = pd.to_datetime(df.datadate.apply(lambda d: str(d.year)+'-06-30'))

# sometimes a company changes its fiscal year and has two annual reports in the same calendar year
# we keep the last annual report in this circumstace

df = df.drop_duplicates(subset=['gvkey','date'],keep='last')  

# the coalesce function implements the following logic:
#   x = a if a exists
#   else x = b if b exists
#   else x = c if c exists
#   ...
# the dataframe that is input to the function should have columns in the following order:
# first column = a (most desired definition)
# second column = b (next most desired definition)
# ... last column = least desired definition (used only if others do not exist)

def coalesce(d) :
    return d.bfill(axis=1).iloc[:, 0]

# Fama-French definition of book equity : shareholders equity + deferred taxes - preferred stock
# Compustat has three preferred stock variables, we prefer pstkrv, then pstkl, then pstk
# we prefer to use seq for shareholders equity, then ceq (common equity) + pstk, then total assets minus total liabilities
# when adding pstk to ceq, we allow for pstk to be missing by filling nans with 0
# we filter to observations with positive book equity

deferredTaxes = df.txditc.fillna(0)    
preferredStock = coalesce(df[['pstkrv','pstkl','pstk']]).fillna(0)                  
seq2 = df.ceq + df.pstk.fillna(0)                
seq3 = np.where((df['at']>=0) & (df['lt']>=0), df['at']-df['lt'], np.nan) 
seq3 = pd.Series(seq3, index=df.index)
shareholdersEquity = coalesce(pd.concat((df.seq,seq2,seq3),axis=1))
df['be'] = shareholdersEquity + deferredTaxes - preferredStock
df = df[df.be>0]
del deferredTaxes, preferredStock, seq2, seq3, shareholdersEquity

# Fama-French definition of operating profitability: revenue - cost of goods sold - SG&A expenses - interest expense divided by equity

costs = df.cogs.fillna(0) + df.xsga.fillna(0) + df.xint.fillna(0)
sales = coalesce(df[['sale','revt']]) 
df['op'] = (sales-costs) / ( df.be + df.mib.fillna(0) )
del costs, sales

# Fama-French definition of investment: percent change in assets
# we filter to observations with positive total assets
# we group by gvkey whenever shifting or calculating changes to avoid errors at the point one firm's data ends and another starts

df = df[df['at']>0]
df['inv'] = df.groupby('gvkey')['at'].pct_change()

# keep the variables we want to use

df = df[['gvkey','date','datadate','be','op','inv']]
df.info()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<class 'pandas.core.frame.DataFrame'>
Int64Index: 401997 entries, 1 to 26494
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   gvkey     401997 non-null  int64         
 1   date      401997 non-null  datetime64[ns]
 2   datadate  401997 non-null  datetime64[ns]
 3   be        401997 non-null  float64       
 4   op        398891 non-null  float64       
 5   inv       368227 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(1)
memory usage: 21.5 MB


# Assign permnos 

We use the CRSP link table. This follows instructions at WRDS.

In [5]:
link = conn.raw_sql(" select distinct gvkey, lpermno as permno, linkdt, linkenddt "
                    " from crsp.Ccmxpf_linktable "
                    " where linktype in ('LU', 'LC') "
                    " and LINKPRIM in ('P', 'C') " )
link['gvkey'] = link.gvkey.astype(int)
link['permno'] = link.permno.astype(int)
link['linkenddt'] = pd.to_datetime(link.linkenddt).fillna(pd.Timestamp('21000101'))
df = df.merge(link,on='gvkey',how='inner')
df = df[(df.datadate>=df.linkdt) & (df.datadate<=df.linkenddt)]
df = df.drop(columns=['gvkey','datadate','linkdt','linkenddt'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274402 entries, 6 to 407280
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    274402 non-null  datetime64[ns]
 1   be      274402 non-null  float64       
 2   op      273171 non-null  float64       
 3   inv     264085 non-null  float64       
 4   permno  274402 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 12.6 MB


# CRSP

We get stock prices and returns from  CRSP (Center for Research in Security Prices) database via WRDS.  There is both monthly daily data.  See http://www.crsp.com/files/data_descriptions_guide_0.pdf for a complete set of variable definitions.  CRSP uses PERMCO as a permanent company identifier and PERMNO as a permanent security identifier.  We will only look at common stock, but some companies have multiple classes of common stock, which means multiple PERMNOs can be associated with a single PERMCO.

In [6]:
freq = 'monthly'
start = '1962-01-01'
crsp_table = 'msf' if freq=='monthly' else 'dsf'
     
df2 = conn.raw_sql("SELECT a.permno, a.permco, a.date, a.ret, abs(a.prc)*a.shrout as me, b.exchcd, b.siccd, b.ticker "
                   " from crsp." + crsp_table + " a inner join crsp.msenames b "
                   " on a.permno=b.permno and a.date between b.namedt and b.nameendt "
                   " and b.exchcd in (1,2,3) and b.shrcd in (10,11) "
                   " where a.date >= '" + start + "' "
                   " order by a.date, a.permco, me ", date_cols=['date'])

for col in ['permno','permco','exchcd'] :
    df2[col] = df2[col].astype(int)

# define market equity as sum of market equities of all permnos associated with a permco
df2['me'] = df2.groupby(['date','permco']).me.transform(sum)

# if there are multiple permnos for a permco, keep only the permno with largest market equity
# works because sql query sorted by market equity within date/permco
df2 = df2.drop_duplicates(subset=['date','permco'],keep='last')
df2 = df2.drop(columns=['permco'])

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3257644 entries, 0 to 291289
Data columns (total 7 columns):
 #   Column  Dtype         
---  ------  -----         
 0   permno  int64         
 1   date    datetime64[ns]
 2   ret     float64       
 3   me      float64       
 4   exchcd  int64         
 5   siccd   float64       
 6   ticker  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 198.8+ MB


# Define delisting returns

This is always done, but there are some different ways to do it.  Here, we follow some of the literature and assign a lower delisting return to Nasdaq stocks than to NYSE/AMEX stocks if the delisting return is missing.

In [7]:
mse = conn.raw_sql(" select permno, dlret, dlstcd " 
                   " from crsp.mse " 
                   " where event='DELIST' and dlstcd>100 "
                   " order by permno")
mse['permno'] = mse.permno.astype(int)
df2 = df2.merge(mse, how='left', on='permno')
del mse
LastObs = df2.permno != df2.permno.shift(-1)                           # True if last date for stock
DLCode = (df2.dlstcd==500) | ( (df2.dlstcd >=520)&(df2.dlstcd<=584) )  # True if delisted for poor performance

df2['dlret'] = np.where(DLCode & df2.dlret.isnull() & df2.exchcd.isin([1,2]), -0.35, df2.dlret )
df2['dlret'] = np.where(DLCode & df2.dlret.isnull() & (df2.exchcd==3), -0.55, df2.dlret )
df2['dlret'] = np.where(df2.dlret.notnull() & df2.dlret<-1,-1,df2.dlret)
df2['ret'] = np.where(LastObs & df2.ret.notnull(), (1+df2.ret)*(1+df2.dlret.fillna(0))-1, df2.ret)
df2['ret'] = np.where(LastObs & df2.ret.isnull(), df2.dlret, df2.ret)
df2 = df2.drop(columns=['dlstcd','dlret'])

# Merge CRSP with Compustat

In [8]:
df = df2.merge(df, on = ['permno','date'], how='left')
df.sort_values(by=['permno','date'],inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3257652 entries, 1066070 to 3255330
Data columns (total 10 columns):
 #   Column  Dtype         
---  ------  -----         
 0   permno  int64         
 1   date    datetime64[ns]
 2   ret     float64       
 3   me      float64       
 4   exchcd  int64         
 5   siccd   float64       
 6   ticker  object        
 7   be      float64       
 8   op      float64       
 9   inv     float64       
dtypes: datetime64[ns](1), float64(6), int64(2), object(1)
memory usage: 273.4+ MB


# Define size and book-to-market

Fama and French use end-of-June market cap to define size.  They use end-of-December market cap to define book-to-market.  We define both size and book-to-market at the end of each June.

In [9]:
month = df.date.apply(lambda d: d.month)
df['size'] = np.where(month==6, df.me, np.nan)
df['bm'] = df.groupby('permno').apply(lambda d: d.be/d.me.shift(6)).values
df['bm'] = np.where(month==6, df.bm, np.nan)
df = df.drop(columns=['be'])

# Fill annual data into months

Use the pandas ffill method to fill forward from Junes into successive months.  Occasionally firms change fiscal years, so filling for more than 12 months can be useful.  On the other hand, we don't want to fill forward if a permno exits the database for years and then returns.  18 months is my arbitrary compromise.

In [10]:
df[['size','bm','op','inv']] = df.groupby('permno')[['size','bm','op','inv']].ffill(limit=18)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3257652 entries, 1066070 to 3255330
Data columns (total 11 columns):
 #   Column  Dtype         
---  ------  -----         
 0   permno  int64         
 1   date    datetime64[ns]
 2   ret     float64       
 3   me      float64       
 4   exchcd  int64         
 5   siccd   float64       
 6   ticker  object        
 7   op      float64       
 8   inv     float64       
 9   size    float64       
 10  bm      float64       
dtypes: datetime64[ns](1), float64(7), int64(2), object(1)
memory usage: 298.2+ MB


## Close WRDS

You should close your WRDS connection when you are finished with it.  

In [11]:
conn.close()

## Save Your Data

If you think you might want to work more with the same data later, you should save it to disk.  Then, you can read it from disk when you want to use it again, which will be quicker than downloading it from WRDS.  The write and read commands are as follows.

    df.to_csv('/path/filename.csv')                                      # writes to disk  
    df = pd.read_csv('/path/filename.csv', date_cols=['date'])           # reads from disk

Here, we mount and save to google drive.

In [12]:
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('/content/drive/My Drive/crsp_compustat_example.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
